In [1]:
import numpy as np
from scipy.optimize import linprog
from ipynb.fs.full.functions import nn2na, find_arc_names, arc_usage

In [2]:
#Import data
NN = np.array([[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],#P1A
               [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],#P1B
               [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],#P2A
               [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],#P2B
               [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],#P3A
               [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],#P3B
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],#S1A
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],#S1B
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],#S2A
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],#S2B
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#C1A
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#C1B
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#C2A
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#C2B
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#C3A
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])#C3B

In [3]:
node_name = ['P1A', 'P1B', 'P2A', 'P2B', 'P3A', 'P3B', 'S1A', 'S1B', 'S2A', 'S2B', 'C1A', 'C1B', 'C2A', 'C2B', 'C3A', 'C3B']

In [4]:
# DATA transforming for optimization:
NA, arc_idxs, arc_idxs_list = nn2na(NN)

### Considerations: 
1. For factories we consider that production must less or equal to their capacity
2. For clients we consider that demand should be exactly attended

Given these two considerations, we must create separate data for linprog to solve it:

In [5]:
NAub = NA.copy()
NAub[6:, :] = 0
print(NAub)

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [6]:
NAeq = NA.copy()
NAeq[:6, :] = 0
print(NAeq)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0 -1  0  0  0 -1  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0 -1  0  0  0 -1  0  0  0  0  1  1  1  0  0  0  0  0  0]
 [ 0 -1  0  0  0 -1  0  0  0 -1  0  0  0  0  0  0  0  0  1  1  1  0  0  0]
 [ 0  0  0 -1  0  0  0 -1  0  0  0 -1  0  0  0  0  0  0  0  0  0  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0

In [7]:
bub = np.array([20, 30, 10, 40, 30, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
beq = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -30, -40, -10, -20, -20, -20])
C = np.array([100, 100, 200, 200, 150, 150, 150, 150, 200, 200, 100, 100, 100, 150, 200, 200, 150, 100, 100, 150, 200, 200, 150, 100])

In [8]:
bounds = tuple([(0, None) for arcs in range(0, NAeq.shape[1])])

In [9]:
# OPTIMIZE:
res = linprog(C, A_eq=NAeq, b_eq=beq, A_ub=NAub, b_ub=bub, bounds=bounds, method='simplex')

In [10]:
# Use functions to express flow related to arc names
arc_names = find_arc_names(arc_idxs_list, node_name)
arc_use = arc_usage(arc_names, res.x.astype(int))
arc_use

{('P1A', 'S1A'): 20,
 ('P1A', 'S2A'): 0,
 ('P1B', 'S1B'): 30,
 ('P1B', 'S2B'): 0,
 ('P2A', 'S1A'): 10,
 ('P2A', 'S2A'): 0,
 ('P2B', 'S1B'): 40,
 ('P2B', 'S2B'): 0,
 ('P3A', 'S1A'): 30,
 ('P3A', 'S2A'): 0,
 ('P3B', 'S1B'): 10,
 ('P3B', 'S2B'): 0,
 ('S1A', 'C1A'): 30,
 ('S1A', 'C2A'): 10,
 ('S1A', 'C3A'): 20,
 ('S1B', 'C1B'): 40,
 ('S1B', 'C2B'): 20,
 ('S1B', 'C3B'): 20,
 ('S2A', 'C1A'): 0,
 ('S2A', 'C2A'): 0,
 ('S2A', 'C3A'): 0,
 ('S2B', 'C1B'): 0,
 ('S2B', 'C2B'): 0,
 ('S2B', 'C3B'): 0}

### CONCLUSION
There are a couple of thing to mention of the output:
1. Because no restriccion of capacity or cost was given to stocks, the model assigns every product to Stock1, and none to S2
2. Because Product 1 and 2 offer and demand are independent from each other, we could solve this problem separatedly and we would reach to the same result.